In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import requests

### Download

In [2]:
city_codes = pd.read_csv('city_codes.csv')
city_codes.head()

,city,code,page_type
0,Alamo,4800,page
1,Blackhawk,4600,page
2,Clayton,5900,page
3,Concord,5701,page
4,Clyde,5702,page


In [3]:
columns = ['City', 'Townhouse-Condo Attached', 'Single-Family Detached', 
           'New Listings', 'Pending Sales',
           'Closed Sales', 'Days on Market Until Sale', 'Median Sales Price',
           'Average Sales Price', 'Percent of Original List Price Received',
           'Inventory of Homes for Sale', 'Year', 'Month']

df = pd.DataFrame(columns=columns)
df

,City,Townhouse-Condo Attached,Single-Family Detached,New Listings,Pending Sales,Closed Sales,Days on Market Until Sale,Median Sales Price,Average Sales Price,Percent of Original List Price Received,Inventory of Homes for Sale,Year,Month


In [4]:
def createCentralStatdf(page_type, month, year, city, city_code, result):
    '''
    INPUT:
    page_type - string, wccar or page
    month - int, 1-12
    year - int, 2011-2021
    city - srting
    city_code - int,
    df - pandas DataFrame
    
    OUTPUT:
    df - pandas DataFrame
    
    '''
    
    central_stat_url = f"http://main.ccartoday.com/index.php/{page_type}/area/{month}/{year}/{city_code}"
    
    table_df = pd.read_html(central_stat_url)
    single_df = table_df[0][1:].transpose()[:3]

    single_df = single_df.rename(columns=single_df.iloc[0]).drop(single_df.index[0])

    single_df['Single-Family Detached'] = 1
    single_df['Townhouse-Condo Attached'] = 0

    condo_df = table_df[1][1:].transpose()[:3]

    condo_df = condo_df.rename(columns=condo_df.iloc[0]).drop(condo_df.index[0])

    condo_df['Single-Family Detached'] = 0
    condo_df['Townhouse-Condo Attached'] = 1
    
    single_df = single_df[1:]
    condo_df = condo_df[1:]

    new_df = pd.concat([single_df, condo_df], ignore_index=True)
    new_df['City'] = city
    new_df['Month'] = month
    new_df['Year'] = year
    
    result[city_code] = new_df


In [75]:
year = 2021

# for month in range(1, 4):
#     for city, city_code in central_city_codes_dict.items():
#         df = createCentralStatdf('page', month, year, city, city_code, df)
        
#     for city, city_code in west_city_codes_dict.keys():
#         df = createCentralStatdf('wccar', month, year, city, city_code, df)

import threading

threads = []
result = {}
for month in range(1, 12):
    for city, city_code in central_city_codes_dict.items():
        x = threading.Thread(target=createCentralStatdf, args=('page', month, year, city, city_code, result))
        threads.append(x)
        x.start()

#     for city, city_code in west_city_codes_dict.items():
#         x = threading.Thread(target=createCentralStatdf, args=('wccar', month, year, city, city_code, result))
#         threads.append(x)
#         x.start()

for _, thread in enumerate(threads):
    thread.join()

for k,v in result.items():
    df = pd.concat([df, v], ignore_index=True)

In [70]:
df.head(20)

,City,Townhouse-Condo Attached,Single-Family Detached,New Listings,Pending Sales,Closed Sales,Days on Market Until Sale,Median Sales Price,Average Sales Price,Percent of Original List Price Received,Inventory of Homes for Sale,Year,Month
0,Blackhawk,0,1,18,NaN,13,21,"$2,400,000","$2,629,461",104.04,13,2021,3
1,Blackhawk,1,0,1,NaN,NaN,0,$0,$0,0,1,2021,3
2,Alamo,0,1,47,NaN,27,14,"$2,350,000","$2,860,506",104.82,37,2021,3
3,Alamo,1,0,NaN,NaN,NaN,0,$0,$0,0,0,2021,3
4,Clayton,0,1,20,NaN,14,6,"$1,021,400","$1,023,254",105.81,11,2021,3
5,Clayton,1,0,NaN,NaN,1,4,"$590,000","$590,000",107.28,0,2021,3
6,Clyde,0,1,NaN,NaN,NaN,0,$0,$0,0,0,2021,3
7,Clyde,1,0,NaN,NaN,NaN,0,$0,$0,0,0,2021,3
8,Pleasant Hill,0,1,46,NaN,37,9,"$1,000,000","$1,023,336",110.11,11,2021,3
9,Pleasant Hill,1,0,8,NaN,14,9,"$651,000","$624,821",104.02,5,2021,3


In [179]:
for year in range(2011, 2021):

    for month in range(1, 13):
        for city in central_city_codes_dict.keys():
            city_code = central_city_codes_dict[city]
            df = createCentralStatdf('page', month, year, city, city_code, df)

        for city in west_city_codes_dict.keys():
            city_code = west_city_codes_dict[city]
            df = createCentralStatdf('wccar', month, year, city, city_code, df)

In [180]:
df.head()

,City,Townhouse-Condo Attached,Single-Family Detached,New Listings,Pending Sales,Closed Sales,Days on Market Until Sale,Median Sales Price,Average Sales Price,Percent of Original List Price Received,Inventory of Homes for Sale,Year,Month
0,Alamo,0,1,23,NaN,11,44,"$2,135,000","$2,369,636",97.56,19,2021,1
1,Alamo,1,0,NaN,NaN,2,50,"$826,500","$826,500",98.77,0,2021,1
2,Blackhawk,0,1,10,NaN,9,64,"$2,700,000","$2,813,144",97.86,13,2021,1
3,Blackhawk,1,0,1,NaN,NaN,0,$0,$0,0,0,2021,1
4,Clayton,0,1,11,NaN,9,13,"$1,028,300","$1,005,356",101.97,7,2021,1


### Cleaning

In [185]:
df['Median Sales Price'] = df['Median Sales Price'].apply(lambda x: x.replace('$', '').replace(',', ''))
df['Median Sales Price'] = df['Median Sales Price'].astype('int32')

In [191]:
df['Average Sales Price'] = df['Average Sales Price'].apply(lambda x: x.replace('$', '').replace(',', ''))
df['Average Sales Price'] = df['Average Sales Price'].astype('int32')

In [198]:
cols = df.drop(['Median Sales Price', 'Average Sales Price', 'City'], axis=1).columns
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')

In [200]:
df.head()

,City,Townhouse-Condo Attached,Single-Family Detached,New Listings,Pending Sales,Closed Sales,Days on Market Until Sale,Median Sales Price,Average Sales Price,Percent of Original List Price Received,Inventory of Homes for Sale,Year,Month
0,Alamo,0,1,23.0,NaN,11.0,44,2135000,2369636,97.56,19,2021,1
1,Alamo,1,0,NaN,NaN,2.0,50,826500,826500,98.77,0,2021,1
2,Blackhawk,0,1,10.0,NaN,9.0,64,2700000,2813144,97.86,13,2021,1
3,Blackhawk,1,0,1.0,NaN,NaN,0,0,0,0.00,0,2021,1
4,Clayton,0,1,11.0,NaN,9.0,13,1028300,1005356,101.97,7,2021,1


In [206]:
df['Year-Month'] = df['Year'].to_st

TypeError: <class 'pandas.core.frame.DataFrame'> is not convertible to datetime

In [199]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8856 entries, 0 to 8855
Data columns (total 13 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   City                                     8856 non-null   object 
 1   Townhouse-Condo Attached                 8856 non-null   int64  
 2   Single-Family Detached                   8856 non-null   int64  
 3   New Listings                             6704 non-null   float64
 4   Pending Sales                            5537 non-null   float64
 5   Closed Sales                             6527 non-null   float64
 6   Days on Market Until Sale                8856 non-null   int64  
 7   Median Sales Price                       8856 non-null   int32  
 8   Average Sales Price                      8856 non-null   int32  
 9   Percent of Original List Price Received  8856 non-null   float64
 10  Inventory of Homes for Sale              8856 no

In [201]:
df.to_csv('Monthly_Stat_Reports.csv', index=False)

In [ ]:
month = {
    1: "January", 
         2: "February", 
         3: "March", 
    4: "April", 
         5: "May", 
         6: "June",
         7: "July", 
         8: "August", 
         9: "September", 
         10: "October", 
         11: "November", 
         12: "December"
        }

df.rename(columns={'Month' : "Month num"},
          inplace=True)

df['Month'] = df.apply(lambda row: month[row["Month num"]], axis=1)

In [2]:
from sqlalchemy import create_engine
from sqlalchemy.types import Integer, Text, String, DateTime, Float

database_filepath = './Monthly_Stat_Reports.db'
engine = create_engine('sqlite:///{}'.format(database_filepath))

sqlite_connection = engine.connect()

In [ ]:
table_name = 'Monthly_Stat_Reports_Cities'
df.to_sql(
    table_name,
    engine,
    if_exists='replace',
    index=False,
    #dtype=dtypes,
    chunksize=500
)

In [ ]:
sqlite_connection.close()